# 린커-쿤다 방법
- https://docs.opencv.org/3.4/d4/dee/tutorial_optical_flow.html
- 그래 이거지

In [12]:
import numpy as np
import cv2 
import argparse

In [13]:
i1 = cv2.imread("frame/frame0.png")
i2 = cv2.imread("frame/frame1.png")
i3 = cv2.imread("frame/frame2.png")
i4 = cv2.imread("frame/frame3.png")
i5 = cv2.imread("frame/frame4.png")
i6 = cv2.imread("frame/frame5.png")
i7 = cv2.imread("frame/frame6.png")

In [48]:
f1 = cv2.cvtColor(i1, cv2.COLOR_BGR2GRAY)
f2 = cv2.cvtColor(i2, cv2.COLOR_BGR2GRAY)

# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )
p0 = cv2.goodFeaturesToTrack(f1, mask = None, **feature_params)

In [57]:
# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15, 15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))


p1, st, err = cv2.calcOpticalFlowPyrLK(f1,f2, p0, None, **lk_params)


good_new = p1[st==1]
good_old = p0[st==1]
transMat, _ = cv2.estimateAffine2D(good_old,good_new)

In [58]:
transMat

array([[ 1.00004662e+00, -4.16074358e-05, -2.56859724e-01],
       [-7.80649164e-05,  1.00005599e+00,  4.07287428e-02]])

In [257]:




cap = cv2.VideoCapture('data/video.MOV')
# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )
# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15, 15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
# Create some random colors
color = np.random.randint(0, 255, (100, 3))

# 확대 행렬
m_big = np.float32([[2, 0, 0], [0, 2, 0]])  

# 시작
for i in range(30*3): cap.read()
frameNm = 30*3-1

# Take first frame and find corners in it
ret, old_frame = cap.read()
# old_frame = cv2.resize(old_frame, None,  None, 2, 2, cv2.INTER_LINEAR)
# old_frame = cv2.pyrUp(old_frame)
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

# 크기
cols, rows = old_gray.shape
transMat0 = None

# 합칠 것
output = np.zeros_like(old_frame)#.astype('float64')
reps = np.ones_like(old_frame).astype('float64')

while(1):
    ret, frame = cap.read()
    frameNm += 1

    if frameNm > 7*30:
        print('종료')
        break

    if not ret:
        print('No frames grabbed!')
        break

    # frame = cv2.resize(frame, None,  None, 2, 2, cv2.INTER_LINEAR)
    # frame = cv2.pyrUp(frame)

    cv2.pyrUp

    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    # Select good points
    if p1 is not None:
        good_new = p1[st==1]
        good_old = p0[st==1]

    # calculate 2D affine transformation matrix 
    transMat1, _ = cv2.estimateAffine2D(good_new, good_old)
    if transMat0 is None: transMat0 = transMat1
    else:
        transMat0 = np.append(transMat1[:, :2]@transMat0[:, :2], (transMat1[:, 2]+transMat0[:, 2]).reshape((2,1)), axis=1)
    new_frame = cv2.warpAffine(frame, transMat0, (rows, cols))
    # print('transMat',transMat, sep='\n')

    # 더하기
    # output += new_frame # 평균
    output = np.array((new_frame,output)).max(axis=0) # 최대

    reps += new_frame.astype('bool').astype('float64')

    # draw the tracks
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
        new_frame = cv2.circle(new_frame, (int(a), int(b)), 5, color[i].tolist(), -1)
    img = cv2.add(new_frame, mask)
    cv2.imshow('frame', cv2.pyrDown(cv2.pyrDown(img)))
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)
cv2.destroyAllWindows()

종료


In [247]:
frame_gray.shape

(1440, 1920)

In [100]:
transMat

array([[ 1.00009300e+00,  3.23334847e-05, -1.83417952e-01],
       [-1.27781977e-05,  9.99969597e-01,  1.25441728e-01]])

In [209]:
# 평균

outimage = (output/reps).astype('uint8')
cv2.imwrite('output3.png',outimage)
cv2.imshow('outimage', outimage)
cv2.waitKey(30)

-1

In [258]:
# max
cv2.imwrite('output8_max_0.5b_pyrUp.png',output)
# cv2.imshow('outimage_max', output)
# cv2.waitKey(30)

True

In [214]:
# (new_frame.astype('bool').astype('float64'))[23]
type(new_frame[0,0,0])

numpy.uint8

In [228]:
np.array((new_frame,output)).max(axis=0).shape

(1440, 1920, 3)

In [229]:
new_frame.shape

(1440, 1920, 3)

In [255]:
cv2.destroyAllWindows()